In [10]:
import numpy as np
import os 
import cv2
import random
import sys 
import csv
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd
import bz2
from collections import defaultdict
from bz2 import BZ2File
from datetime import datetime
from PIL import Image
from sklearn.metrics import roc_curve, accuracy_score
from sklearn.metrics import auc, confusion_matrix
import sys
from scikit_posthocs import posthoc_ttest
sys.path.append('.')
from src.modules import *

In [11]:
def statistics(results_all, bs_num=50):
    
    np.random.seed(10)
    
    perf_models = {}
    for j in range(len(results_all)):
        perf_models[f'perf_bs{j}'] = []
    
    for k in range(bs_num):
        class0idx = np.where(results_all[10][0]==0)[0]
        class1idx = np.where(results_all[10][0]==1)[0]
        idx0 = np.random.choice(class0idx, len(class0idx))
        idx1 = np.random.choice(class1idx, len(class1idx))
        idx = list(idx0) + list(idx1)
        
        i = 1
        for j in range(len(perf_models)):
            results_epoch = results_all[j]

            if len(results_epoch[0]) < 3000:
                class0idx = np.where(results_all[j][0]==0)[0]
                class1idx = np.where(results_all[j][0]==1)[0]
                idx0 = np.random.choice(class0idx, len(class0idx))
                idx1 = np.random.choice(class1idx, len(class1idx))
                idx = list(idx0) + list(idx1)

            _gt = results_epoch[0][idx]
            _pred = results_epoch[1][idx]
            _attr = results_epoch[2][:, idx]

            val_es_acc, val_es_auc, val_aucs_by_attrs, val_dpds, val_eods, val_between_group_disparity = evalute_comprehensive_perf(_pred,
                                                                                                                                _gt,
                                                                                                                                _attr)

            perfi = [val_es_acc[i], val_es_auc[i]]
            perfi.extend(val_aucs_by_attrs[i])
            perfi.append(val_dpds[i])
            perfi.append(val_eods[i])
            perfi.extend(val_between_group_disparity[i])
#                 perf_bs_proposed[f'attr{i}'].append(perfi)

            perf_models[f'perf_bs{j}'].append(perfi)
    
    attr_pvalues = []
    
    num_metrics = len(perf_models['perf_bs0'][0])
    
    for k in range(num_metrics):
        vpaired = []
        for j in range(len(perf_models.keys())):
            vpaired.append(np.array(perf_models[f'perf_bs{j}'])[:, k])

        vpaired = np.array(vpaired)
        vpaired = vpaired[:, ~np.isnan(vpaired).any(axis=0)]

        pvalues = posthoc_ttest(vpaired)

        attr_pvalues.append(pvalues)
            
    return attr_pvalues

In [12]:
# # transform adversarial results

folder = '/data/home/shim/pyspace/others/pyspace/ICLR_30k/Harvard-DR30k_results/results_odir_FINAL/dr_slo_fundus_adversarial'

outcome_adv = {}

idx = 1   
attr = 'gender'
modality = f'dr_slo_fundus_{attr}'

modality_folder = f'{folder}/{modality}'

bestresults = []
epochresults = []
for f in os.listdir(modality_folder):
    if f.endswith('.csv'):
        paths = pd.read_csv(os.path.join(modality_folder, f))['  path']
        bestresults.append([f, paths[0].split('/')[-1].split('auc')[0]])
    else:
        epochresults.append(f)
pairedresults = {}
for i, v in enumerate(bestresults):
    for p in epochresults:
        if v[1] in p:
            pairedresults[i] = [v[0], p]

for k in pairedresults.keys():
    v = pairedresults[k]

    resultpd = pd.read_csv(os.path.join(modality_folder, v[0]))
    results = {}
    attrgroupnums = {}
    for name, data in resultpd.items():
        results[name.strip()] = data.values[-1]
        if 'group' in name:
            attri = name.split('_')[1]
            if attri not in attrgroupnums.keys():
                attrgroupnums[attri] = 1
            else:
                attrgroupnums[attri] += 1
    modelname = v[0].split('_')[1]

    vs = [results[f'esacc_attr{idx}'], results['acc'], results[f'esauc_attr{idx}'], results['auc']]
    vs = vs + [results[f'auc_attr{idx}_group{j}'] for j in range(attrgroupnums[f'attr{idx}'])]
    vs = vs + [results[f'dpd_attr{idx}'], results[f'eod_attr{idx}'], 
               results[f'std_group_disparity_attr{idx}'],
               results[f'max_group_disparity_attr{idx}']]

    npzdata = np.load(os.path.join(modality_folder, v[1], 'pred_gt_best_epoch.npz'))
    test_gt = npzdata['test_gt']
    test_pred = npzdata['test_pred']
    test_attr = npzdata['test_attr']
    vs_epoch = [test_gt, test_pred, test_attr]

#         print(len(test_gt), len(test_pred), len(test_attr), os.path.join(modality_folder, v[1], 'pred_gt_best_epoch.npz'))

    outcome_adv[modelname] = (vs, vs_epoch)


In [28]:
# # transform proposed model results

folder = '/data/home/shim/pyspace/others/pyspace/ICLR_30k/Harvard-DR30k_results/results_odir_FINAL/'

outcome_fis = {}

idx = 1   
attr = 'gender'
modality = f'dr_slo_fundus_{attr}_fis'

modality_folder = f'{folder}/{modality}'

bestresults = []
epochresults = []
for f in os.listdir(modality_folder):
    if f.endswith('.csv'):
        paths = pd.read_csv(os.path.join(modality_folder, f))['  path']
        bestresults.append([f, paths[0].split('/')[-1].split('auc')[0]])
    else:
        epochresults.append(f)
pairedresults = {}
for i, v in enumerate(bestresults):
    for p in epochresults:
        if v[1] in p:
            pairedresults[i] = [v[0], p]

for k in pairedresults.keys():
    v = pairedresults[k]

    resultpd = pd.read_csv(os.path.join(modality_folder, v[0]))
    results = {}
    attrgroupnums = {}
    for name, data in resultpd.items():
        results[name.strip()] = data.values[-1]
        if 'group' in name:
            attri = name.split('_')[1]
            if attri not in attrgroupnums.keys():
                attrgroupnums[attri] = 1
            else:
                attrgroupnums[attri] += 1
    modelname = v[0].split('_')[1]

    vs = [results[f'esacc_attr{idx}'], results['acc'], results[f'esauc_attr{idx}'], results['auc']]
    vs = vs + [results[f'auc_attr{idx}_group{j}'] for j in range(attrgroupnums[f'attr{idx}'])]
    vs = vs + [results[f'dpd_attr{idx}'], results[f'eod_attr{idx}'], 
               results[f'std_group_disparity_attr{idx}'],
               results[f'max_group_disparity_attr{idx}']]

    npzdata = np.load(os.path.join(modality_folder, v[1], 'pred_gt_best_epoch.npz'))
    test_gt = npzdata['test_gt']
    test_pred = npzdata['test_pred']
    test_attr = npzdata['test_attr']
    vs_epoch = [test_gt, test_pred, test_attr]

#         print(len(test_gt), len(test_pred), len(test_attr), os.path.join(modality_folder, v[1], 'pred_gt_best_epoch.npz'))

    outcome_fis[modelname] = (vs, vs_epoch)


In [31]:
# statistic analysis
modality = 'slo_fundus' #'slo_fundus or oct_bscans'
folder = '/data/home/shim/pyspace/others/pyspace/ICLR_30k/Harvard-DR30k_results/results_odir_FINAL'


modality_folder = f'{folder}/dr_{modality}_gender'

model_order = ['vgg', 'swin', 'resnet', 'convnext', 'densenet', 'efficientnet', 'ViT-B', 
               'vgg_oversample','swin_oversample', 'resnet_oversample', 'convnext_oversample', 
               'densenet_oversample', 'efficientnet_oversample', 'ViT-B_oversample']

bestresults = []
epochresults = []
for f in os.listdir(modality_folder):
    if f.endswith('.csv'):
        print(f)
        paths = pd.read_csv(os.path.join(modality_folder, f))['  path']
        bestresults.append([f, paths[0].split('/')[-1].split('auc')[0]])
    else:
        epochresults.append(f)
        
pairedresults = {}
modelindex = {}
for i, v in enumerate(bestresults):
    for p in epochresults:
        if v[1] in p:
            pairedresults[i] = [v[0], p]
            modelname = v[0].split('_')[1]
            if 'oversample' in v[0]:
                modelname = modelname + '_oversample'
            modelindex[modelname] = i

best_results_all = []
column_names = []
rownames = []
pred_results_all = []

for k in model_order:
    if k not in modelindex.keys():
        continue
    k = modelindex[k]
    v = pairedresults[k]
    resultpd = pd.read_csv(os.path.join(modality_folder, v[0]))
    results = {}
    attrgroupnums = {}
    for name, data in resultpd.items():
        results[name.strip()] = data.values[-1]
        if 'group' in name:
            attri = name.split('_')[1]
            if attri not in attrgroupnums.keys():
                attrgroupnums[attri] = 1
            else:
                attrgroupnums[attri] += 1
    modelname = v[0].split('_')[1]
    if 'oversample' in v[0]:
        modelname = modelname + '_oversample'
    
    rownames.append(modelname)
    print(modelname)
    npzdata = np.load(os.path.join(modality_folder, v[1], 'pred_gt_best_epoch.npz'))
    test_gt = npzdata['test_gt']
    test_pred = npzdata['test_pred']
    test_attr = npzdata['test_attr']
    
    i = 1
    vs = [results[f'esacc_attr{i}'], results['acc'], results[f'esauc_attr{i}'], results['auc']]
    vs = vs + [results[f'auc_attr{i}_group{j}'] for j in range(attrgroupnums[f'attr{i}'])]
    vs = vs + [results[f'dpd_attr{i}'], results[f'eod_attr{i}'], 
               results[f'std_group_disparity_attr{i}'],
               results[f'max_group_disparity_attr{i}']]
    best_results_all.append(vs)

    vsname = ['esacc', 'acc', 'esauc', 'auc']
    vsname = vsname + [f'auc_group{j}' for j in range(attrgroupnums[f'attr{i}'])]
    vsname = vsname + ['dpd', 'eod', 'std_group_disparity', 'max_group_disparity']
    column_names = vsname

    pred_results_all.append([test_gt, test_pred, test_attr])


# append adversarial outcome
modeladv_order = ['vgg', 'swin', 'resnet', 'convnext', 'densenet', 'efficientnet', 'ViT-B']
for modelname in modeladv_order:
    modelname_subfix = f'{modelname}_adv'
    rownames.append(modelname_subfix)
    print(modelname_subfix)
    best_results_all.append(outcome_adv[modelname][0])
    pred_results_all.append(outcome_adv[modelname][1])
    
# append fis outcome
modelfis_order = ['efficientnet', 'ViT-B']
for modelname in modelfis_order:
    modelname_subfix = f'{modelname}_fis'
    rownames.append(modelname_subfix)
    print(modelname_subfix)
    best_results_all.append(outcome_fis[modelname][0])
    pred_results_all.append(outcome_fis[modelname][1])
    
# write best outcome    
writer = pd.ExcelWriter(f"{folder}/dr_{modality}_race_combined_outcome_odir.xlsx", engine="xlsxwriter")

pdattri = pd.DataFrame(best_results_all, 
             index=rownames, 
             columns=column_names)
pdattri.to_excel(writer, sheet_name='Gender')

    
attr_pvalues = statistics(pred_results_all)

rowidx = 0
vsname = ['esacc', 'acc', 'esauc', 'auc']
vsname = vsname + [f'auc_group{j}' for j in range(len(attr_pvalues))]
vsname = vsname + ['dpd', 'eod', 'std_group_disparity', 'max_group_disparity']

for idx, pvalues in enumerate(attr_pvalues):

    rownames_prefix = [str(v)+'@'+vsname[idx] for v in rownames]

    pvdf = pd.DataFrame(np.array(pvalues), index=rownames_prefix, columns=rownames_prefix)

    pvdf.to_excel(writer, sheet_name='gender_statistics', startrow=rowidx)
    rowidx += len(pvalues) + 2
    
writer.close()
# pairedresults

best_densenet_slo_fundus_male_oversample.csv
best_swin_slo_fundus_gender.csv
best_ViT-B_slo_fundus_male_oversample.csv
best_efficientnet_slo_fundus_gender.csv
best_vgg_slo_fundus_male_oversample.csv
best_convnext_slo_fundus_male_oversample.csv
best_resnet_slo_fundus_gender.csv
best_densenet_slo_fundus_gender.csv
best_vgg_slo_fundus_gender.csv
best_ViT-B_slo_fundus_gender.csv
best_resnet_slo_fundus_male_oversample.csv
best_efficientnet_slo_fundus_male_oversample.csv
best_convnext_slo_fundus_gender.csv
best_swin_slo_fundus_male_oversample.csv
vgg
swin
resnet
convnext
densenet
efficientnet
ViT-B
vgg_oversample
swin_oversample
resnet_oversample
convnext_oversample
densenet_oversample
efficientnet_oversample
ViT-B_oversample
vgg_adv
swin_adv
resnet_adv
convnext_adv
densenet_adv
efficientnet_adv
ViT-B_adv
efficientnet_fis
ViT-B_fis


In [7]:
# attr_pvalues = statistics(pred_results_all)

# rowidx = 0
# vsname = ['esacc', 'acc', 'esauc', 'auc']
# vsname = vsname + [f'auc_group{j}' for j in range(len(attr_pvalues))]
# vsname = vsname + ['dpd', 'eod', 'std_group_disparity', 'max_group_disparity']

# for idx, pvalues in enumerate(attr_pvalues):

#     rownames_prefix = [str(v)+'@'+vsname[idx] for v in rownames]

#     pvdf = pd.DataFrame(np.array(pvalues), index=rownames_prefix, columns=rownames_prefix)

#     pvdf.to_excel(writer, sheet_name='gender_statistics', startrow=rowidx)
#     rowidx += len(pvalues) + 2
    
# writer.close()

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/xlsxwriter/workbook.py:368: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
